**The objective of this project is to classify an individual's pulmonary records to determine the presence or absence of pneumonia. To achieve this goal, we will develop a classification model utilizing transfer learning, specifically leveraging the VGG16 architecture.**

In [4]:
# importing library

import tensorflow as tf 
import os
import numpy as np
from glob import glob

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten
from keras.models import Model

In [5]:
print(tf.__version__)

2.10.0


In [9]:
# Let's Download the DataBase
# Note that our Dataset from Kaggle named Pneumonia Data. I downloaded it and save on my local disk. Note that you van can
# download directly on kaggle and save in your drive 

data_path = "C:/Users/nguim/OneDrive/Bureau/Learning/Project ML/Datasets/pneumonia_data/chest_xray"

In [10]:
# Train and test directory
train_dir = os.path.join(data_path, "train")
test_dir = os.path.join(data_path, 'test')

In [14]:
# Let's specify the size of images we will use:
IMG_SHAPE=(224,224,3)

In [15]:
# Let's define the base model: we instaniate the VGG16 model
base_model=VGG16(
    input_shape=IMG_SHAPE, 
    include_top=False, 
    weights='imagenet'
)

base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
# let's freeze the models parameters
base_model.trainable=False

In [18]:
base_model.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [41]:
# as we can see the shape of the output of our model is not pretty good. we need to fit it
x=Flatten()(base_model.output)
prediction=Dense(2, activation='softmax')(x)

In [42]:
prediction

<KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'dense_6')>

In [43]:
base_model.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [54]:
base_model.input

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_2')>

In [55]:
# Now let's create our own model base on base_model
model = Model(
    inputs=base_model.input, 
    outputs=prediction
)

model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [56]:
model.output

<KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'dense_6')>

In [62]:
# Let's compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accurary'])

In [63]:
# Data augmentation
train_datagen=ImageDataGenerator(rescale=1./255, 
                                 shear_range=0.2, 
                                 zoom_range=0.2, 
                                 horizontal_flip=True
                                 )

test_datagen=ImageDataGenerator(rescale=1./255)

trainning_set=train_datagen.flow_from_directory(train_dir, 
                                                target_size=(224, 224), 
                                                batch_size=32, 
                                                class_mode='categorical'
                                                )

test_set=test_datagen.flow_from_directory(test_dir, 
                                          target_size=(224, 224), 
                                                batch_size=32, 
                                                class_mode='categorical'
                                                )


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [64]:
# Le't apply and train our model

model.fit(trainning_set, epochs=1, validation_data=test_set)

C:\Users\nguim\AppData\Local\Temp\ipykernel_13196\3787086972.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(trainning_set, epochs=1, validation_data=test_set)


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.